In [25]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
#apparently installing alongside the python-Levenshtein module can increase performance


Inspired by  

https://www.datacamp.com/community/tutorials/fuzzy-string-python

The Levenshtein distance is a metric to measure how apart are two sequences of words. In other words, it measures the minimum number of edits that you need to do to change a one-word sequence into the other. These edits can be insertions, deletions or substitutions. This metric was named after Vladimir Levenshtein, who originally considered it in 1965.

In [16]:
Str1 = "Apple Inc."
Str2 = "apple Inc"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
print(Ratio)

95


In [17]:
Str1 = "Los Angeles Lakers"
Str2 = "L.A. Lakers"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
Token_Sort_Ratio = fuzz.token_sort_ratio(Str1.lower(),Str2.lower())
print(Ratio)
print(Partial_Ratio)
print(Token_Sort_Ratio)

62
73
64


fuzz.partial_ratio() is capable of detecting that both strings are referring to the Lakers. Thus, it yields 100% similarity. The way this works is by using an "optimal partial" logic. In other words, if the short string has length k and the longer string has the length m, then the algorithm seeks the score of the best matching length-k substring.

Nevertheless, this approach is not foolproof. What happens when the strings comparison the same, but they are in a different order?

In [43]:
Str1 = "united states v. nixon"
Str2 = "Nixon v. United States"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
Token_Sort_Ratio = fuzz.token_sort_ratio(Str1,Str2)
print(Ratio)
print(Partial_Ratio)
print(Token_Sort_Ratio)

59
74
100


The fuzz.token functions have an important advantage over ratio and partial_ratio. They tokenize the strings and preprocess them by turning them to lower case and getting rid of punctuation. In the case of fuzz.token_sort_ratio(), the string tokens get sorted alphabetically and then joined together. After that, a simple fuzz.ratio() is applied to obtain the similarity percentage. This allows cases such as court cases in this example to be marked as being the same.

Still, what happens if these two strings are of widely differing lengths? Thats where fuzz.token_set_ratio() comes in.

In [46]:
Str1 = "The supreme court case of Nixon vs The United States"
Str2 = "Nixon v. United States"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
Token_Sort_Ratio = fuzz.token_sort_ratio(Str1,Str2)
Token_Set_Ratio = fuzz.token_set_ratio(Str1,Str2)
print(Ratio)
print(Partial_Ratio)
print(Token_Sort_Ratio)
print(Token_Set_Ratio)

57
77
58
95


fuzz.token_set_ratio() takes a more flexible approach than fuzz.token_sort_ratio(). Instead of just tokenizing the strings, sorting and then pasting the tokens back together, token_set_ratio performs a set operation that takes out the common tokens (the intersection) and then makes fuzz.ratio() pairwise comparisons between the following new strings:

s1 = Sorted_tokens_in_intersection  
s2 = Sorted_tokens_in_intersection + sorted_rest_of_str1_tokens  
s3 = Sorted_tokens_in_intersection + sorted_rest_of_str2_tokens  
The logic behind these comparisons is that since Sorted_tokens_in_intersection is always the same, the score will tend to go up as these words make up a larger chunk of the original strings or the remaining tokens are closer to each other.

In [158]:
df_short= pd.read_excel("./test_data/data_short.xlsx")
df_long=pd.read_excel("./test_data/data_long.xlsx")


In [159]:
df_short.head()

,Unnamed: 0,address,city,country,dob,first_name,full_address,full_name,gender,last_name,member_id,middle_name,suffix,title
0,0,84724 Nicole Villages Suite 945,Leetown,New Zealand,2009-11-05,Deborah,"84724 Nicole Villages Suite 945, Leetown, New ...",Deborah Nunez,F,Nunez,5102279799502,NaN,NaN,Dr.
1,1,658 Aaron Vista Apt. 239,Mistyborough,Micronesia,1958-03-25,Nathan,"658 Aaron Vista Apt. 239, Mistyborough, Micron...",Nathan Baker,M,Baker,2719238400725,NaN,NaN,Mr.
2,2,66108 Vasquez Course,Jeffreymouth,Serbia,1961-07-01,Ronald,"66108 Vasquez Course, Jeffreymouth, Serbia",Ronald Smith,M,Smith,5344277168281,NaN,NaN,Mr.
3,3,117 Wood Turnpike Apt. 562,North Christopher,Montenegro,1951-12-14,Douglas,"117 Wood Turnpike Apt. 562, North Christopher,...",Douglas Stephanie Ramirez,M,Ramirez,9262734362101,Stephanie,NaN,Mr.
4,4,7483 Nguyen Square,North Benjamin,Germany,1950-12-06,Alex,"7483 Nguyen Square, North Benjamin, Germany",Alex Curry,M,Curry,2504631179862,NaN,NaN,Mr.


In [160]:
df_long.head()

,Unnamed: 0,address,city,country,dob,first_name,full_address,full_name,gender,last_name,member_id,middle_name,suffix,title
0,0,98006 Daniel Causeway,Morenoborough,Palestinian Territory,1940-10-13,Cynthia,"98006 Daniel Causeway, Morenoborough, Palestin...",Cynthia Brandy Brown,F,Brown,4005263871981,Brandy,NaN,Mrs.
1,1,3871 Stevens Lane Apt. 513,East Margaret,Seychelles,1982-02-03,John,"3871 Stevens Lane Apt. 513, East Margaret, Sey...",John Campbell,M,Campbell,5593277759078,NaN,NaN,Mr.
2,2,707 Nichole Run,New Jerrybury,France,2017-09-13,Tara,"707 Nichole Run, New Jerrybury, France",Tara Amanda Manning,F,Manning,9253704032896,Amanda,NaN,Mrs.
3,3,29214 Christopher Lodge,Lake Andrew,Isle of Man,1955-11-13,Michaela,"29214 Christopher Lodge, Lake Andrew, Isle of Man",Michaela Brock,F,Brock,4666408668950,NaN,NaN,Dr.
4,4,31860 Earl Stravenue Suite 296,Vickiburgh,Belgium,1974-02-17,Peter,"31860 Earl Stravenue Suite 296, Vickiburgh, Be...",Peter Perez,M,Perez,4289840277902,NaN,NaN,Mr.


In [161]:
df_short['fuzzy']=df_short['full_name'].str.lower()
df_short.set_index('fuzzy')
list_to_check_against= list(set(df_short['fuzzy']))
print(len(list_to_check_against))

500


In [162]:
df_long['fuzzy']=df_long['full_name'].str.lower()
df_short.set_index('fuzzy')
print(df_long.shape)
data_to_check= list(set(df_long['fuzzy']))
print(len(data_to_check))

(2000, 15)
1983


In [163]:
matches = []
for item in data_to_check:
    if type(item) != str: # Checking base case. 
        pass
    else:
        match = process.extractOne(item, list_to_check_against, scorer=fuzz.token_sort_ratio) #Returns tuple of best match and percent fit.
        matches.append([item,match[0],match[1]])
        

    
        

In [164]:
possible_matches=[]
possible_matches= [x for x in matches if x[2]>95]
possible_matches

[['katherine smith', 'katherine smith', 100],
 ['olivia smith', 'smith, olivia', 100],
 ['michael taylor', 'michael taylor', 100],
 ['george collins', 'george collins', 100],
 ['david williams', 'david williams', 100],
 ['christopher brown', 'christopher brown', 100],
 ['andrew hernandez', 'andrew hernandez', 100],
 ['daniel gonzales', 'daniel gonzales', 100],
 ['mary hernandez', 'mary hernandez', 100],
 ['jessica alvarez', 'jessica alvarez', 100],
 ['james martin', 'james martin', 100]]

In [165]:

output_list=[]
for x in possible_matches:
    key1=df_short.loc[df_short['fuzzy'] == x[1]]['member_id'].tolist()
    key2=df_long.loc[df_long['fuzzy'] == x[0]]['member_id'].tolist()
    output_list.append([x[1], x[0], x[2], key1, key2])
output= pd.DataFrame(output_list, columns=['short_list','long_list','perc_match','key_short','key_long'])

In [166]:
df_short.loc[df_short['fuzzy'] == 'smith, olivia']['member_id']

432    4358287106424
Name: member_id, dtype: int64

In [167]:
output

,short_list,long_list,perc_match,key_short,key_long
0,katherine smith,katherine smith,100,[7563384088130],[8986386973416]
1,"smith, olivia",olivia smith,100,[4358287106424],[1067119077629]
2,michael taylor,michael taylor,100,[5847830015911],[2836812545211]
3,george collins,george collins,100,[1306491549530],[5179019038436]
4,david williams,david williams,100,[229720631152],[3922931562549]
5,christopher brown,christopher brown,100,[1970815105834],[502887664677]
6,andrew hernandez,andrew hernandez,100,[358339665507],[2340862914892]
7,daniel gonzales,daniel gonzales,100,[140312516230],[7993593208263]
8,mary hernandez,mary hernandez,100,[7534575761576],[6492089346744]
9,jessica alvarez,jessica alvarez,100,[9250994501583],[9790621226361]
